## 字符串插值
摘抄自scala官网

### 简介
自2.10.0版本开始，Scala提供了一种新的机制来根据数据生成字符串：字符串插值。字符串插值允许使用者将变量引用直接插入处理过的字面字符中。如下例：
```scala
val name="James"
println(s"Hello,$name")//Hello,James
```

在上例中， `s”Hello,$name”` 是待处理字符串字面，编译器会对它做额外的工作。待处理字符串字面通过“号前的字符来标示（例如：上例中是s）

### 用法
Scala 提供了三种创新的字符串插值方法：s,f 和 raw.

#### s 字符串插值器
在任何字符串前加上s，就可以直接在串中使用变量了。你已经见过这个例子：
```scala
val name="James"
println(s"Hello,$name")//Hello,James 此例中，$name嵌套在一个将被s字符串插值器处理的字符串中。插值器知道在这个字符串的这个地方应该插入这个name变量的值，以使输出字符串为Hello,James。使用s插值器，在这个字符串中可以使用任何在处理范围内的名字。
```

字符串插值器也可以处理任意的表达式。例如：
```scala
println(s"1+1=${1+1}") //将会输出字符串1+1=2。任何表达式都可以嵌入到${}中
```

#### f 插值器
在任何字符串字面前加上 f，就可以生成简单的格式化串，功能相似于其他语言中的 printf 函数。当使用 f 插值器的时候，所有的变量引用都应当后跟一个printf-style格式的字符串，如%d。看下面这个例子：
```scala
val height=1.9d
val name="James"
println(f"$name%s is $height%2.2f meters tall")//James is 1.90 meters tall f 插值器是类型安全的。如果试图向只支持 int 的格式化串传入一个double 值，编译器则会报错。例如：

val height:Double=1.9d

scala>f"$height%4d"
<console>:9: error: type mismatch;
 found : Double
 required: Int
```

#### raw 插值器
除了对字面值中的字符不做编码外，raw 插值器与 s 插值器在功能上是相同的。如下是个被处理过的字符串：
```scala
scala>s"a\nb"
res0:String=
a
b 
//这里，s 插值器用回车代替了\n。而raw插值器却不会如此处理。

scala>raw"a\nb"
res1:String=a\nb 
//当不想输入\n被转换为回车的时候，raw 插值器是非常实用的。
```

除了以上三种字符串插值器外，使用者可以自定义插值器。

## 可插入字符串
摘抄自《scala程序设计（第2版）》————3.13 可插入字符串

我们在 1.4 节已经介绍了什么是可插入字符串(interpolated string),这里将对其进行更深层次地分析。

假如某一字符串的形式为 `s"foo ${bar}"` ,那么表达式 bar 将会被转化为字符串并被插入到原字符串中的 $"{bar}" 的位置中。假如表达式 bar 返回的不是字符串,而是某一类型的实例,那么只要该实例中定义了 toString 方法,系统便会调用该方法将该实例转化成字符串。如果 bar 表达式返回值无法转换成字符串,那么程序将会报错。

如果 bar 是一个变量引用,那么可以省略字符串中的大括号。如下所示:
```scala
val name = "Buck Trends"
println(s"Hello, $name")
```

如果想在可插入字符串中输入美元符,那么请连续输入两个美元符 `$$` 。

Scala 中存在两类可插入字符串。第一类采用 printf 格式,这类可插入字符串以 f 为前缀。第二类也被称为“原生的”可插入字符串,这类字符串并不会对像 \n 这样的逃逸字符串进行扩展。

可插入字符串以 f 为前缀:
```scala
val gross = 100000F
val net = 64000F
val percent = (net / gross) * 100
println(f"$$${gross}%.2f vs. $$${net}%.2f or ${percent}%.1f%%")
```
最后一行代码将输出下列结果:
```
$100000.00 vs. $64000.00 or 64.0%
```

采用 printf 格式时,Scala 会调用 Java 的 Formatter 类。这类字符串中嵌入的表达式与之前代码使用的语法相同,均为 `$"{...}"` ,不过在编写时 printf 格式指令与 `${...}` 之间不应有空格。

在该示例中,为了打印出一个美元符号,我们使用了两个美元符 `$$` ;同时使用了两个百分号 `%%` 以打印出一个百分号。表达式 `${gross}%.2f` 会格式化 gross 的变量值,保留其小数点后两位数字。

可插入字符串中的变量类型必须与其格式吻合,为此 Scala 会执行一些隐式转化。下面列出的代码试图在浮点数语境中使用 Int 表达式,Scala 允许这种操作并会在整数的小数点后添加两个 0。而第二个表达式则尝试将 Double 类型值展现为 Int 类型,不过这次尝试会导致编译错误:
```scala
scala> val i = 200
i: Int = 200

scala> f"${i}%.2f"
res4: String = 200.00

scala> val d = 100.22
d: Double = 100.22)

scala> f"${d}%2d"
<console>:9: error: type mismatch;
found
: Double
required: Int
```

顺 便 提 一 下, 调 用 Java 静 态 方 法 String.format将按照 printf 的格式对字符串格式化。该方法的输入参数包含了格
式字符串以及一组用于替代格式字符串的变量列表。 String.format 方法还有另外一个版本,该版本中的第一个参数代表字符串的区域设置(该参数为 Locale 类型)。

Scala 编译器会在某些语境中对 Java 字符串进行封装并提供一些额外的方法,这些定义在 scala.collection.immutable.StringLike 中。这些额外提供的方法中包含了一个叫作 format的实例方法。你可以对一个格式化字符串调用该方法并传入需要插入到该字符串中的参数列表。具体如下:
```scala
scala> val s = "%02d: name = %s".format(5, "Dean Wampler")
s: String = "05: name = Dean Wampler"
```

在该示例中,我们希望能输出两位数整数并在小数点添加两个零。最后一类内置的字符串插入器被称为“原生”(raw)插入器,该插入器不会对控制字符进行扩展,具体请参阅下面两个示例。
```scala
scala> s"123\n$name\n456"
res0: String =
123
Dean Wampler
456

scala> raw"123\n$name\n456"
res1: String = 123\nDean Wampler\n456
```

最后提一下,其实我们可以自定义字符串插入器,不过在此之前,我们需要掌握更多的关于隐式转换(implicit)的知识。如果想了解具体细节,请参阅 5.3.1 节。